In [5]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("You are using GPU")
else:
    print("You are using CPU")

You are using GPU


In [6]:
df = pd.read_csv('data_processing/csv_file/output_file.csv')
class ProcessData(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.iloc[:, 1:].values
        self.labels = dataframe.iloc[:, 0].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {
            'data': torch.tensor(self.data[idx], dtype=torch.float32, device=device),
            'label': torch.tensor(self.labels[idx], dtype=torch.long, device=device)
        }
        return sample

keypoint_dataset = ProcessData(df)
print(keypoint_dataset.__len__())

5896


In [7]:
import model as m
model = m.SimpleNN4().to(device)
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

dataloader = DataLoader(keypoint_dataset, batch_size = 128, shuffle = True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Model's state_dict:
fc1.weight 	 torch.Size([80, 126])
fc1.bias 	 torch.Size([80])
fc2.weight 	 torch.Size([64, 80])
fc2.bias 	 torch.Size([64])
fc3.weight 	 torch.Size([32, 64])
fc3.bias 	 torch.Size([32])
fc4.weight 	 torch.Size([20, 32])
fc4.bias 	 torch.Size([20])
fc5.weight 	 torch.Size([10, 20])
fc5.bias 	 torch.Size([10])
fc6.weight 	 torch.Size([8, 10])
fc6.bias 	 torch.Size([8])


In [8]:
num_epochs = 200
previous_loss = float('inf')  # Initialize with a high value

for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = batch['data']
        labels = batch['label']

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    if loss < previous_loss:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        previous_loss = loss  # Update the previous loss

torch.save(model.state_dict(), "models/5_9_newest.pth")

Epoch [1/200], Loss: 1.9282
Epoch [2/200], Loss: 1.9262
Epoch [3/200], Loss: 1.2516
Epoch [5/200], Loss: 1.1833
Epoch [6/200], Loss: 0.9999
Epoch [8/200], Loss: 0.9226
Epoch [10/200], Loss: 0.8508
Epoch [11/200], Loss: 0.7996
Epoch [19/200], Loss: 0.6269
Epoch [33/200], Loss: 0.4649
Epoch [36/200], Loss: 0.4299
Epoch [44/200], Loss: 0.3171
Epoch [60/200], Loss: 0.2769
Epoch [67/200], Loss: 0.1838
Epoch [100/200], Loss: 0.1723
Epoch [134/200], Loss: 0.1401
Epoch [146/200], Loss: 0.0701
